### Imports

In [1]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import LabelEncoder
from preprocess_data import preprocess_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import SGDClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-09-21 15:26:14.730160: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 15:26:14.730205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 15:26:14.731469: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-

### Functions

In [2]:
def load_data(model_name):
    """
    Load and preprocess data for text classification using a pre-trained sentence-transformer model.

    Parameters:
    model_name (str): The name of the pre-trained sentence-transformer model to use for encoding text descriptions.

    Returns:
    tuple: A tuple containing:
        - X_train_full (numpy array): Original training data embeddings.
        - X_test (numpy array): Original test data embeddings.
        - X_train_full_normalized (numpy array): Normalized training data embeddings.
        - X_test_normalized (numpy array): Normalized test data embeddings.
        - y_train_full (numpy array): Training labels.
        - y_test (numpy array): Test labels.
    """
    data = preprocess_data()

    data['num_genres'] = data['genre'].apply(len)
    data = data[data['num_genres'] == 1]

    descriptions = data['description_processed'].tolist()
    genres = data['genre'].tolist()

    descriptions = data['description_processed'].tolist()
    genres = data['genre'].tolist()
    # Encode the genres as numerical labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(genres)

    # Load a pre-trained sentence-transformer model to convert text to embeddings
    model = SentenceTransformer(model_name)

    # Convert descriptions to vector embeddings
    X = model.encode(descriptions, show_progress_bar=True)

    # Split the data into train and test sets
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize the embeddings for cosine similarity
    X_train_full_normalized = normalize(X_train_full, axis=1, norm='l2')
    X_test_normalized = normalize(X_test, axis=1, norm='l2')

    return X_train_full,X_test, X_train_full_normalized,X_test_normalized,y_train_full,y_test


In [ ]:
def random_sampling(iterations,clf,X_test,y_test):
    """
    Perform active learning with random sampling to iteratively improve a classifier's performance.

    Parameters:
    iterations (int): The number of active learning iterations to perform.
    clf (classifier object): The classifier to be trained. 
    X_test (numpy array): The test data on which to evaluate the classifier's performance.
    y_test (numpy array): The true labels for the test data.

    Returns:
    None: The function prints accuracy after each iteration and a final classification report.
    """
    accuracy_scores = []

    for iteration in range(iterations):
        # Incrementally train the classifier with the current training set
        clf.partial_fit(X_train, y_train)

        # Predict and evaluate performance on the test set
        y_pred = clf.predict(X_test)
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        
        print(f"Iteration {iteration + 1}: Accuracy = {accuracy_scores[-1]:.4f}")

        # Random sampling of new data points
        sample_size = min(sample_size, len(X_pool))
        selected_indices = np.random.choice(len(X_pool), size=sample_size, replace=False)
        X_sample = X_pool[selected_indices]

        # Handle potential differences in data type or format
        try:
            y_sample = y_pool[selected_indices]
        except:
            y_sample = y_pool.iloc[selected_indices]
        
        # Update the training set with new samples
        X_train = np.vstack([X_train, X_sample])
        y_train = np.concatenate([y_train, y_sample])

        # Remove the selected samples from the pool
        X_pool = np.delete(X_pool, selected_indices, axis=0)
        y_pool = np.delete(y_pool, selected_indices, axis=0)

        # Stop criteria based on the pool size or maximum training size
        if len(X_pool) == 0 or len(X_pool) < sample_size or len(X_train) >= 60000:
            break

    # Print the final classification report
    print(classification_report(y_test, y_pred))

In [ ]:
def ANN_as_clustering(iterations,dynamic_clusters,embedding_dim,X_train_full_normalized,clf,budget_per_iteration,uncertainty_threshold,y_train_full,index,X_test_normalized,y_test):
    """
    Perform active learning using Approximate Nearest Neighbors (ANN) as a clustering strategy.

    Parameters:
    iterations (int): The number of active learning iterations to perform.
    dynamic_clusters (int): The initial number of clusters, which increases with each iteration.
    embedding_dim (int): The dimensionality of the embedding space.
    X_train_full_normalized (numpy array): The normalized full training data embeddings.
    clf (classifier object): The classifier to be trained incrementally.
    budget_per_iteration (int): The maximum number of samples to be added to the training set per iteration.
    uncertainty_threshold (float): The threshold for selecting uncertain samples within each cluster.
    y_train_full (numpy array): The labels corresponding to the full training data.
    index (faiss.IndexFlatL2): The FAISS index used for clustering and searching.
    X_test_normalized (numpy array): The normalized test data embeddings.
    y_test (numpy array): The true labels for the test data.

    Returns:
    None: The function prints the test accuracy after each iteration and the final classification report.
    """
    # Active Learning Loop
    for iteration in range(iterations):
        # Adjust the number of clusters dynamically
        
        num_clusters = dynamic_clusters + iteration  # Increase clusters gradually

        # Initialize FAISS clustering
        clustering = faiss.Clustering(embedding_dim, num_clusters)
        clustering.verbose = False
        clustering.niter = 50

        # Convert remaining indices to the appropriate format
        remaining_data = np.array([X_train_full_normalized[i] for i in remaining_indices]).astype('float32')
        index_flat = faiss.IndexFlatL2(embedding_dim)
        clustering.train(remaining_data, index_flat)

        # Get cluster assignments
        D, cluster_assignments = index_flat.search(remaining_data, 1)
        
        # Convert FAISS centroids to numpy array
        centroids = faiss.vector_to_array(clustering.centroids).reshape(num_clusters, embedding_dim)

        # Select samples from each cluster using a refined strategy
        selected_indices = []
        for cluster in range(num_clusters):
            cluster_indices = [i for i, label in zip(remaining_indices, cluster_assignments) if label == cluster]
            if cluster_indices:
                # Select the most uncertain samples within each cluster
                cluster_data = np.array([X_train_full_normalized[i] for i in cluster_indices]).astype('float32')
                probs = clf.predict_proba(cluster_data)
                uncertainty = 1 - np.max(probs, axis=1)

                # Calculate diversity by distance from the centroid
                distances_from_centroid = np.linalg.norm(cluster_data - centroids[cluster], axis=1)
                combined_scores = uncertainty + distances_from_centroid

                # Prioritize samples with high uncertainty and diversity
                sorted_indices = np.argsort(-combined_scores)
                num_samples = min(int(len(cluster_indices) * uncertainty_threshold), len(cluster_indices))
                selected_cluster_indices = [cluster_indices[i] for i in sorted_indices[:num_samples]]
                selected_indices.extend(selected_cluster_indices)

        # Ensure unique samples and limit to the budget
        selected_indices = list(set(selected_indices))
        if len(selected_indices) > budget_per_iteration:
            selected_indices = selected_indices[:budget_per_iteration]

        # Add selected samples to the training set
        X_train = np.vstack((X_train, X_train_full_normalized[selected_indices]))
        y_train = np.concatenate((y_train, np.array(y_train_full)[selected_indices]))

        # Remove selected samples from the pool
        remaining_indices = list(set(remaining_indices) - set(selected_indices))

        # Update FAISS Index with new training data
        index.add(X_train_full_normalized[selected_indices])

        # Incrementally train the classifier with new samples
        clf.partial_fit(X_train, y_train)

        # Evaluate the classifier on the test set after each iteration
        y_pred = clf.predict(X_test_normalized)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Iteration {iteration + 1}: Test Accuracy = {accuracy:.4f}")
    
    print(classification_report(y_test, y_pred))

In [102]:
def train_nn(model, X_train, y_train, optimizer, criterion, batch_size=64, epochs=5):
    """
    Train a neural network model using the provided training data, optimizer, and loss function.

    Parameters:
    model (torch.nn.Module): The neural network model to be trained.
    X_train (torch.Tensor): The input training data.
    y_train (torch.Tensor): The labels corresponding to the training data.
    optimizer (torch.optim.Optimizer): The optimizer used to update the model parameters.
    criterion (torch.nn.Module): The loss function used to compute the error between predictions and true labels.
    batch_size (int, optional): The number of samples per batch during training. Default is 64.
    epochs (int, optional): The number of times to iterate over the entire training dataset. Default is 5.

    Returns:
    None: The function prints the loss after each epoch.
    """
    model.train()

    # Create dataset and dataloader
    dataset = TensorDataset(X_train, y_train)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in loader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(loader):.4f}')

In [103]:
def evaluate_nn(device, model, X_test, batch_size=64):
    """
    Evaluate a neural network model on the test dataset and return the predicted labels.

    Parameters:
    device (torch.device): The device (CPU or GPU) on which to perform the evaluation.
    model (torch.nn.Module): The neural network model to be evaluated.
    X_test (numpy array): The test data on which to evaluate the model.
    batch_size (int, optional): The number of samples per batch during evaluation. Default is 64.

    Returns:
    numpy array: An array of predicted labels for the test data.
    """
    model.eval()

    # Convert data to PyTorch tensor
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    
    # Create dataset and dataloader
    dataset = TensorDataset(X_test_tensor)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    predictions = []
    with torch.no_grad():
        for inputs in loader:
            outputs = model(inputs[0])
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    return np.array(predictions)

In [ ]:
def AL_with_nn(iterations,dynamic_clusters,embedding_dim,X_train_full_normalized,device,model,uncertainty_threshold,budget_per_iteration,y_train_full,optimizer, criterion,X_test_normalized,y_test):
    """
    Perform active learning with neural networks using dynamic clustering and uncertainty sampling.

    Parameters:
    iterations (int): The number of active learning iterations to perform.
    dynamic_clusters (int): The initial number of clusters, which increases with each iteration.
    embedding_dim (int): The dimensionality of the embedding space.
    X_train_full_normalized (numpy array): The normalized full training data embeddings.
    device (torch.device): The device (CPU or GPU) on which to perform computations.
    model (torch.nn.Module): The neural network model to be trained.
    uncertainty_threshold (float): The threshold for selecting uncertain samples within each cluster.
    budget_per_iteration (int): The maximum number of samples to be added to the training set per iteration.
    y_train_full (numpy array): The labels corresponding to the full training data.
    optimizer (torch.optim.Optimizer): The optimizer used to update the model parameters.
    criterion (torch.nn.Module): The loss function used to compute the error between predictions and true labels.
    X_test_normalized (numpy array): The normalized test data embeddings.
    y_test (numpy array): The true labels for the test data.

    Returns:
    None: The function prints the test accuracy after each iteration and the final classification report.
    """
    # Active Learning Loop
    for iteration in range(iterations):
        # Adjust the number of clusters dynamically
        num_clusters = min(dynamic_clusters + iteration, len(remaining_indices) // 2)

        # Initialize FAISS clustering
        clustering = faiss.Clustering(embedding_dim, num_clusters)
        clustering.verbose = False
        clustering.niter = 50

        # Convert remaining indices to the appropriate format
        remaining_data = np.array([X_train_full_normalized[i] for i in remaining_indices]).astype('float32')
        index_flat = faiss.IndexFlatL2(embedding_dim)
        clustering.train(remaining_data, index_flat)

        # Get cluster assignments
        _, cluster_assignments = index_flat.search(remaining_data, 1)
        cluster_assignments = cluster_assignments.flatten()

        # Convert FAISS centroids to numpy array
        centroids = faiss.vector_to_array(clustering.centroids).reshape(num_clusters, embedding_dim)

        # Select samples from each cluster using a refined strategy
        selected_indices = []
        for cluster in range(num_clusters):
            cluster_indices = [i for i, label in zip(remaining_indices, cluster_assignments) if label == cluster]
            if cluster_indices:
                # Select the most uncertain samples within each cluster
                cluster_data = np.array([X_train_full_normalized[i] for i in cluster_indices]).astype('float32')
                cluster_data_tensor = torch.tensor(cluster_data, dtype=torch.float32).to(device)

                # Get model predictions and uncertainties
                model.eval()
                with torch.no_grad():
                    outputs = model(cluster_data_tensor)
                    probs = nn.functional.softmax(outputs, dim=1)
                    uncertainty = 1 - torch.max(probs, dim=1).values.cpu().numpy()

                # Calculate diversity by distance from the centroid
                distances_from_centroid = np.linalg.norm(cluster_data - centroids[cluster], axis=1)
                combined_scores = uncertainty + distances_from_centroid

                # Prioritize samples with high uncertainty and diversity
                sorted_indices = np.argsort(-combined_scores)
                num_samples = min(int(len(cluster_indices) * uncertainty_threshold), len(cluster_indices))
                selected_cluster_indices = [cluster_indices[i] for i in sorted_indices[:num_samples]]
                selected_indices.extend(selected_cluster_indices)

        # Ensure unique samples and limit to the budget
        selected_indices = list(set(selected_indices))
        if len(selected_indices) > budget_per_iteration:
            selected_indices = selected_indices[:budget_per_iteration]

        # Add selected samples to the training set
        X_train = np.vstack((X_train, X_train_full_normalized[selected_indices]))
        y_train = np.concatenate((y_train, np.array(y_train_full)[selected_indices]))

        # Remove selected samples from the pool
        remaining_indices = list(set(remaining_indices) - set(selected_indices))

        # Update FAISS Index with new training data
        index_flat.add(X_train_full_normalized[selected_indices])

        # Convert updated training data to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

        # Incrementally train the neural network with the new samples
        train_nn(model, X_train_tensor, y_train_tensor, optimizer, criterion, batch_size=32, epochs=5)

        # Evaluate the neural network on the test set after each iteration
        X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32).to(device)
        y_pred = evaluate_nn(device,model, X_test_tensor)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Iteration {iteration + 1}: Test Accuracy = {accuracy:.4f}")

    y_pred_final = evaluate_nn(device,model, X_test_normalized)
    final_accuracy = accuracy_score(y_test, y_pred_final)
    print(f"Final Test Accuracy after all iterations: {final_accuracy:.4f}")

### Load Data and Set Parameters

In [ ]:
X_train_full,X_test, X_train_full_normalized,X_test_normalized,y_train_full,y_test = load_data('sentence-transformers/all-mpnet-base-v2')

In [217]:
# Active Learning parameters
initial_train_size = 100
iterations = 20
sample_size = 250
uncertainty_threshold = 0.2
budget_per_iteration = 250    

#Selecting initial training set randomly
np.random.seed(42)
pool_indices = np.random.choice(len(X_train_full_normalized), initial_train_size, replace=False)
X_train = X_train_full_normalized[pool_indices]
y_train = np.array(y_train_full)[pool_indices]

# Initialize the FAISS Index for Cosine Similarity
faiss.omp_set_num_threads(12)
embedding_dim = X_train_full_normalized.shape[1]
index = faiss.IndexFlatIP(embedding_dim)  # Inner product index for cosine similarity
index.add(X_train_full_normalized)  # Add all normalized vectors to the index

# Remaining pool of indices
remaining_indices = list(set(range(len(X_train_full_normalized))) - set(pool_indices))

Batches: 100%|██████████| 762/762 [01:04<00:00, 11.77it/s]


### Simple ML Model

In [ ]:
# Initialize the classifier
clf = SGDClassifier(loss='log_loss', random_state=42)
clf.partial_fit(X_train, y_train, classes=np.unique(y_train_full))

#### Random Sampling

In [219]:
X_pool, X_train, y_pool, y_train = train_test_split(X_train_full, y_train_full, test_size=0.00255, random_state=42)

In [220]:
random_sampling(iterations,clf,X_test,y_test)

Iteration 1: Accuracy = 0.3765
Iteration 2: Accuracy = 0.4044
Iteration 3: Accuracy = 0.4874
Iteration 4: Accuracy = 0.5798
Iteration 5: Accuracy = 0.5769
Iteration 6: Accuracy = 0.5820
Iteration 7: Accuracy = 0.6081
Iteration 8: Accuracy = 0.6122
Iteration 9: Accuracy = 0.6225
Iteration 10: Accuracy = 0.6138
Iteration 11: Accuracy = 0.6218
Iteration 12: Accuracy = 0.6007
Iteration 13: Accuracy = 0.6282
Iteration 14: Accuracy = 0.6323
Iteration 15: Accuracy = 0.6286
Iteration 16: Accuracy = 0.6274
Iteration 17: Accuracy = 0.6366
Iteration 18: Accuracy = 0.6354
Iteration 19: Accuracy = 0.6327
Iteration 20: Accuracy = 0.6245
              precision    recall  f1-score   support

           0       0.43      0.39      0.41       134
           1       0.40      0.09      0.14        47
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        10
           4       0.53      0.74      0.62      1371
           5       0.00      0.00      0.00 

/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined an

#### ANN As Clustering

In [33]:
dynamic_clusters = 5

ANN_as_clustering(iterations,dynamic_clusters,embedding_dim,X_train_full_normalized,clf,budget_per_iteration,uncertainty_threshold,y_train_full,index,X_test_normalized,y_test)

              precision    recall  f1-score   support

           0       0.53      0.35      0.42       134
           1       0.57      0.09      0.15        47
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        10
           4       0.58      0.69      0.63      1371
           5       0.00      0.00      0.00        35
           6       0.69      0.78      0.73      2233
           7       0.67      0.04      0.07        53
           8       0.00      0.00      0.00        15
           9       0.00      0.00      0.00         5
          10       0.65      0.67      0.66       406
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        23
          14       0.33      0.01      0.02        80
          15       0.89      0.29      0.44        55
          16       0.00      0.00      0.00         3
          17       0.29    

/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/student/.cache/pypoetry/virtualenvs/dataanalysisvisualizationfiles-qa5NmZKI-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined an

### NN Model

In [99]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super (NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [109]:
# Define device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = X_train_full_normalized.shape[1]
hidden_size = 128  # You can adjust the number of neurons
num_classes = len(np.unique(y_train_full))
learning_rate = 0.001

# Initialize the neural network, loss function, and optimizer
model = NN(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [108]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

In [104]:
AL_with_nn(iterations,dynamic_clusters,embedding_dim,X_train_full_normalized,device,model,uncertainty_threshold,budget_per_iteration,y_train_full,optimizer,criterion,X_test_normalized,y_test)

WARNING clustering 14 points to 5 centroids: please provide at least 195 training points


Epoch [1/5], Loss: 2.9518
Epoch [2/5], Loss: 2.9193
Epoch [3/5], Loss: 2.8840
Epoch [4/5], Loss: 2.8416
Epoch [5/5], Loss: 2.8002
Iteration 1: Test Accuracy = 0.3500


/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 6 centroids: please provide at least 234 training points


Epoch [1/5], Loss: 2.7380
Epoch [2/5], Loss: 2.6956
Epoch [3/5], Loss: 2.6078
Epoch [4/5], Loss: 2.5587
Epoch [5/5], Loss: 2.4624
Iteration 2: Test Accuracy = 0.3745


/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 7 centroids: please provide at least 273 training points


...

/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 7 centroids: please provide at least 273 training points


Epoch [1/5], Loss: 0.1051
Epoch [2/5], Loss: 0.1019
Epoch [3/5], Loss: 0.0967
Epoch [4/5], Loss: 0.0986
Epoch [5/5], Loss: 0.0923
Iteration 18: Test Accuracy = 0.5309


/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 7 centroids: please provide at least 273 training points


Epoch [1/5], Loss: 0.0934
Epoch [2/5], Loss: 0.0887
Epoch [3/5], Loss: 0.0801
Epoch [4/5], Loss: 0.0792
Epoch [5/5], Loss: 0.0731
Iteration 19: Test Accuracy = 0.5332
Epoch [1/5], Loss: 0.0720
Epoch [2/5], Loss: 0.0711
Epoch [3/5], Loss: 0.0700
Epoch [4/5], Loss: 0.0684
Epoch [5/5], Loss: 0.0666
Iteration 20: Test Accuracy = 0.5325


/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
WARNING clustering 14 points to 7 centroids: please provide at least 273 training points
/tmp/ipykernel_5349/1057497763.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
